In [ ]:
# ===============================
# Stage 2 — ESG Keywords Extraction
# ===============================
!pip -q install openai pandas numpy nltk pyarrow

from google.colab import drive; drive.mount('/content/drive', force_remount=True)

import os, re, json, numpy as np, pandas as pd
from pathlib import Path
import nltk; nltk.download("punkt", quiet=True)
import nltk; nltk.download("punkt_tab", quiet=True)
from nltk.tokenize import sent_tokenize
from openai import OpenAI

# ---------- Config ----------
COMPANIES_ROOT = "<<Google Drive Link>>"  # I uploaded the ESG report PDF files on Google Drive. Each company has 1 folder
UNIFIED_FILE   = "<<Google Drive Link>>/unified_keywords.csv"

# Subset of company folders --> to use for filtering company folder for analysis
FOLDER_SELECT  = "006-010"

YEAR_LIST   = []                   # e.g., [2022, 2023]; [] = no filter
REPORT_TYPE = []                   # e.g., ["ESG", "AR" (Annual Report),"10K" (10K Filings)]; [] = no filter

EMB_MODEL    = "text-embedding-3-small"
VERIFY_MODEL = "gpt-4o-mini"
THRESH, HIGH_SIM, DELTA_MARGIN = 0.33, 0.50, 0.05
VERIFY_CAP, MAX_KW = 2500, 15
MIN_SENT_LEN, MAX_SENT_PER_DOC = 25, 5000

# Resumable: if True, skip any doc whose Stage-2 JSON already exists
ONLY_MISSING = True

DIVISIONS = [
    ("A","Agriculture, Forestry & Fishing"), ("B","Mining"), ("C","Construction"),
    ("D","Manufacturing"), ("E","Transportation & Public Utilities"),
    ("F","Wholesale Trade"), ("G","Retail Trade"),
    ("H","Finance, Insurance & Real Estate"), ("I","Services"), ("J","Public Administration"),
]

# ---------- OpenAI ----------
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAPI")
client = OpenAI()

# ---------- Ontology / profiles ----------
def load_concepts(path=UNIFIED_FILE):
    df = pd.read_csv(path, usecols=["pillar","canonical_concept","keyword"]).dropna()
    concepts = {}
    for (p, cc), g in df.groupby(["pillar","canonical_concept"]):
        kw = sorted(set(map(str, g["keyword"])))[:MAX_KW]
        concepts.setdefault(p, {})[cc] = kw
    return concepts
CONCEPTS = load_concepts()

def build_profiles(concepts):
    texts, index = [], []
    for p, d in concepts.items():
        for cc, kws in d.items():
            texts.append(f"PILLAR:{p}. CONCEPT:{cc}. Related terms: {', '.join(kws)}")
            index.append((p, cc))
    return texts, index

# ---------- Embeddings / utils ----------
def embed_batched(texts, model=EMB_MODEL, bsz=512):
    if not texts: return np.zeros((0,1536))
    out=[]
    for i in range(0, len(texts), bsz):
        r = client.embeddings.create(model=model, input=texts[i:i+bsz])
        out.extend([d.embedding for d in r.data])
    return np.array(out)

def cosine_matrix(A,B):
    A = A/(np.linalg.norm(A,axis=1,keepdims=True)+1e-12)
    B = B/(np.linalg.norm(B,axis=1,keepdims=True)+1e-12)
    return A @ B.T

def _resolve(path_str: str, base_dir: Path) -> str:
    if not path_str or not isinstance(path_str, str):
        return ""
    p = Path(path_str)
    if p.is_absolute():
        return str(p)
    return str((base_dir / p).resolve())

def read_sentences(row):
    base_dir = Path(row.get("__base_dir__", ""))  # per-company stage dir (esg_stage or esg_stage1)
    s = row.get("sentences_path"); t = row.get("text_path")
    sents=[]
    if s and isinstance(s,str):
        s_abs = _resolve(s, base_dir)
        if os.path.exists(s_abs):
            df = pd.read_parquet(s_abs, columns=["text"])
            sents = [str(x).strip() for x in df["text"].tolist()]
    if not sents and t and isinstance(t,str):
        t_abs = _resolve(t, base_dir)
        if os.path.exists(t_abs):
            txt=open(t_abs,"r",encoding="utf-8").read(); txt=re.sub(r"\s+"," ",txt).strip()
            sents=[x.strip() for x in sent_tokenize(txt)]
    sents=[x for x in sents if len(x)>=MIN_SENT_LEN]
    return sents[:MAX_SENT_PER_DOC] if MAX_SENT_PER_DOC else sents

# ---------- Agents (SIC division, pillar verifier, referee) ----------
DIV_LIST = "\n- ".join([f"{c} — {n}" for c,n in DIVISIONS])
def classify_division(sentences, take=60):
    ctx=" ".join(sentences[:take])[:6000]
    prompt=(f"Choose ONE SIC Division from:\n- {DIV_LIST}\n"
            'Return ONLY JSON: {"code":"A-J","name":"...","confidence":0.0-1.0}.\n\nTEXT:'+ctx)
    r=client.chat.completions.create(model=VERIFY_MODEL,temperature=0,
        response_format={"type":"json_object"},messages=[{"role":"user","content":prompt}])
    try:
        o=json.loads(r.choices[0].message.content)
        return o.get("code","I"), o.get("name","Services"), float(o.get("confidence",0))
    except:
        return "I","Services",0.0

class PillarAgent:  # The agent to classify the pillar might consume a lot of resources
    def __init__(self,p): self.p=p
    def verify(self,sent,concept):
        prompt=("Return ONLY JSON {\"supports\":true|false,\"reason\":\"<=15 words\"}.\n"
                "Does SENTENCE support CONCEPT for PILLAR?\n"
                f"PILLAR:{self.p}\nCONCEPT:{concept}\nSENTENCE:{sent}")
        r=client.chat.completions.create(model=VERIFY_MODEL,temperature=0,
            response_format={"type":"json_object"},messages=[{"role":"user","content":prompt}],max_tokens=60)
        try:
            o=json.loads(r.choices[0].message.content); return bool(o.get("supports",False)), o.get("reason","")
        except:
            return False,"parse_error"

class Referee: # The referee agent to ensure no duplicates between E/S/G sentences might consume a lot of resources
    def decide(self,sent,a,b):
        prompt=('Return ONLY JSON {"assign_to":"environmental|social|governance|none","reason":"<=15 words"}.\n'
                f"SENTENCE:{sent}\nA:{a['pillar']}:{a['concept']} ({a['reason']})\nB:{b['pillar']}:{b['reason']}")
        r=client.chat.completions.create(model=VERIFY_MODEL,temperature=0,
            response_format={"type":"json_object"},messages=[{"role":"user","content":prompt}],max_tokens=60)
        try:
            o=json.loads(r.choices[0].message.content); return o.get("assign_to","none")
        except:
            return "none"

# ---------- Quant detection ----------  --> This detection is to avoid page number, year, date, and other singular number as 'quantitative sentence'
FOOTNOTE_LEAD = re.compile(r"^\s*(?:\(?\d+\)?|\[\d+\]|[¹²³⁴⁵⁶⁷⁸⁹])\s+")
PAGINATION = re.compile(r"\b(page|pp\.|p\.|appendix|annex|exhibit|figure|table|chapter|section|see page)\b", re.I)
TOC_LINE   = re.compile(r"\.{3,}\s*\d{1,4}$")
DATE_HINT  = re.compile(r"\b(19|20)\d{2}\b|\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec|fy|q[1-4])\b", re.I)
NUM_PAT    = re.compile(r"[+-]?\d{1,3}(?:[,\s]\d{3})*(?:\.\d+)?|[+-]?\d+(?:\.\d+)?")
METRIC_HINT = re.compile(
    r"""(?ix)
    (tco2e?|co2e?|ghg|emission|scope\s*[123]|carbon|methane|ch4|
     kwh|mwh|gwh|wh|gj|mj|energy|electricity|renewable|
     m3|m2|m²|lit(?:re|er)s?|water|withdrawals?|discharge|
     kg|t|tonnes?|metric\s*tons?|waste|recycled|landfill|
     incident[s]?|injur(?:y|ies)|fatalit(?:y|ies)|spills?|leaks?|
     fine[s]?|penalt(?:y|ies)|hours|training|turnover|absenteeism|
     diversity|board|pay|remuneration|audit|briber|ethic|
     usd|aud|eur|cad|sgd|yen|cny|inr|zar|mxn|brl|sek|nok|dkk|
     us\$|a\$|\$|£|¥|€)
    """)
WINDOW_METRIC = re.compile(r"(?:^|\b)(?:rate|intensity|emissions?|energy|water|waste|injur|incident|fine|penalt|spent|invested|allocated|revenue|cost|capex)\b", re.I)

def clean_for_quant(s:str)->str:
    s = FOOTNOTE_LEAD.sub("", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

QF_SHOTS = [
  {"role":"user","content":"Is this quantitative? Years/quarters/pages alone are NOT quantitative.\nText: We aim for net zero by 2050."},
  {"role":"assistant","content":json.dumps({"quantitative":False})},
  {"role":"user","content":"Text: Scope 1 emissions decreased to 0.9 MtCO2e in 2024."},
  {"role":"assistant","content":json.dumps({"quantitative":True})},
  {"role":"user","content":"Text: In FY2023 we enhanced our safety culture."},
  {"role":"assistant","content":json.dumps({"quantitative":False})},
  {"role":"user","content":"Text: Renewable electricity reached 60% of total consumption."},
  {"role":"assistant","content":json.dumps({"quantitative":True})},
  {"role":"user","content":"Text: See page 12 for 2023 results."},
  {"role":"assistant","content":json.dumps({"quantitative":False})},
]

def is_quantitative(sentence:str)->bool:
    s = clean_for_quant(sentence)
    if TOC_LINE.search(s): return False
    if PAGINATION.search(s) and not METRIC_HINT.search(s): return False
    has_num    = bool(NUM_PAT.search(s))
    has_metric = bool(METRIC_HINT.search(s))
    if has_metric and has_num: return True
    if DATE_HINT.search(s) and not has_metric: return False
    if has_num and not (has_metric or WINDOW_METRIC.search(s)): return False
    if has_num or has_metric:
        prompt = ('Return ONLY JSON {"quantitative":true|false}. '
                  'Rule: years/quarters/pages alone are NOT quantitative; require a non-date metric.\n'
                  f"Text: {s}")
        msgs = [*QF_SHOTS, {"role":"user","content":prompt}]
        try:
            r = client.chat.completions.create(model=VERIFY_MODEL, temperature=0,
                                               response_format={"type":"json_object"},
                                               messages=msgs, max_tokens=12)
            o = json.loads(r.choices[0].message.content)
            return bool(o.get("quantitative", False))
        except:
            return False
    return False

# ---------- Granularity ----------  --> LLM Agent is used to differentiate whether a quantative sentence belongs to general or specific. I think this part use a lot of tokens & processing time
SPECIFIC_PAT = re.compile(
    r"""(?ix)\b(from|to|versus|vs\.?|compared|baseline|yoy|year[-\s]?on[-\s]?year|
       reduc(?:e|ed)|increas(?:e|ed)|improv(?:e|ed)|decreas(?:e|ed)|
       drop(?:ped)?|rose|grew|cut|achiev(?:e|ed))\b""")

GR_SHOTS = [
  {"role":"user","content":"Quantitative: general or specific?\nText: Reduce emissions 30% by 2030."},
  {"role":"assistant","content":json.dumps({"granularity":"General"})},
  {"role":"user","content":"Text: Scope 2 fell from 120 GWh to 95 GWh vs a 2022 baseline."},
  {"role":"assistant","content":json.dumps({"granularity":"Specific"})},
  {"role":"user","content":"Text: Injury rate improved to less than 5% per million hours."},
  {"role":"assistant","content":json.dumps({"granularity":"General"})},
  {"role":"user","content":"Text: Water withdrawals decreased by 18% year-on-year from 10.5 to 8.6 million m3."},
  {"role":"assistant","content":json.dumps({"granularity":"Specific"})},
]

def granularity_q(sentence:str)->str:
    s = clean_for_quant(sentence)
    if NUM_PAT.search(s) and SPECIFIC_PAT.search(s): return "Specific"
    prompt = ('Return ONLY JSON {"granularity":"General|Specific"}. '
              "Specific = explicit change/baseline/comparison; General = single level/target without comparison.\n"
              f"Text: {s}")
    msgs = [*GR_SHOTS, {"role":"user","content":prompt}]
    try:
        r = client.chat.completions.create(model=VERIFY_MODEL, temperature=0,
                                           response_format={"type":"json_object"},
                                           messages=msgs, max_tokens=12)
        o = json.loads(r.choices[0].message.content)
        g = str(o.get("granularity","General")).lower()
        return "Specific" if g.startswith("spec") else "General"
    except:
        return "General"

# ---------- Per-document (label the same SIC Division within one folder) ----------
def analyze_doc(sentences, precomputed_div=None):
    if not sentences:
        return {"sic_division":None,"sentences":0,"coverage":{},"status_by_pillar":{}}

    #  Use precomputed (company-level) SIC if provided; else classify once here
    if precomputed_div:
        div_code, div_name = precomputed_div
    else:
        div_code, div_name, _ = classify_division(sentences)

    profiles, idx = build_profiles(CONCEPTS)
    S = cosine_matrix(embed_batched(sentences), embed_batched(profiles, bsz=64))

    # boost likely-quantitative sentences slightly
    num_hint = np.array([bool(NUM_PAT.search(s) or METRIC_HINT.search(s)) for s in sentences])
    if num_hint.any(): S[num_hint,:] = np.minimum(S[num_hint,:] * 1.08, 1.0)

    cand = np.argwhere(S>=THRESH)
    agents = {p:PillarAgent(p) for p in CONCEPTS.keys()}
    ref    = Referee()

    claims, borderline = [], []
    for si,ci in cand:
        si,ci=int(si),int(ci); sim=float(S[si,ci]); p,c=idx[ci]; s=sentences[si]
        (claims if sim>=HIGH_SIM else borderline).append((si,ci,sim,p,c,s))
    if len(borderline)>VERIFY_CAP:
        borderline = sorted(borderline, key=lambda x:x[2], reverse=True)[:VERIFY_CAP]

    accepted=[]
    for si,ci,sim,p,c,s in claims:
        accepted.append({"sid":si,"sent":s,"pillar":p,"concept":c,"sim":sim,"reason":"high_similarity"})
    for si,ci,sim,p,c,s in borderline:
        ok,reason = agents[p].verify(s,c)
        if ok: accepted.append({"sid":si,"sent":s,"pillar":p,"concept":c,"sim":sim,"reason":reason})

    # anti double-count per sentence
    assigned={}
    tmp={}
    for a in accepted: tmp.setdefault(a["sid"],[]).append(a)
    for sid,arr in tmp.items():
        if len(arr)==1: assigned[sid]=arr[0]
        else:
            arr=sorted(arr,key=lambda x:x["sim"],reverse=True)
            if arr[0]["sim"]-arr[1]["sim"]>=DELTA_MARGIN: assigned[sid]=arr[0]
            else:
                who=ref.decide(sentences[sid],arr[0],arr[1])
                if who!="none":
                    pick=next((x for x in arr if x["pillar"]==who),None)
                    if pick: assigned[sid]=pick

    # coverage with Quant-wins (Specific > General > Qualitative)
    coverage = {p:{cc:{
        "count":0,"examples":[],
        "qual_examples":[],"qgen_examples":[],"qspec_examples":[],
        "n_qual":0,"n_qgen":0,"n_qspec":0,
        "labels":set(),"final_label":"None"
    } for cc in CONCEPTS[p]} for p in CONCEPTS}

    for _, cl in assigned.items():
        p, cc, s = cl["pillar"], cl["concept"], cl["sent"]
        st = coverage[p][cc]; st["count"] += 1
        if is_quantitative(s):
            g = granularity_q(s)
            if g=="Specific":
                st["n_qspec"] += 1;
                if len(st["qspec_examples"])<3: st["qspec_examples"].append(s)
                st["labels"].add("Quantitative (Specific)")
            else:
                st["n_qgen"]  += 1;
                if len(st["qgen_examples"])<3: st["qgen_examples"].append(s)
                st["labels"].add("Quantitative (General)")
        else:
            st["n_qual"] += 1
            if len(st["qual_examples"])<3: st["qual_examples"].append(s)
            st["labels"].add("Qualitative")

    for p,d in coverage.items():
        for cc,st in d.items():
            if   "Quantitative (Specific)" in st["labels"]: st["final_label"]="Quantitative (Specific)"
            elif "Quantitative (General)" in st["labels"]:  st["final_label"]="Quantitative (General)"
            elif "Qualitative" in st["labels"]:             st["final_label"]="Qualitative"
            else:                                           st["final_label"]="None"
            st["labels"] = list(st["labels"])

    status = {p:{
        "satisfied":[cc for cc,st in d.items() if st["count"]>0],
        "missing"  :[cc for cc,st in d.items() if st["count"]==0]
    } for p,d in coverage.items()}

    return {"sic_division":{"code":div_code,"name":div_name},
            "sentences":len(sentences),
            "coverage":coverage,
            "status_by_pillar":status}

# ---------- Folder-selection ----------
def parse_folder_select(selector: str):
    if not selector or not str(selector).strip():
        return None
    keep = set()
    for tok in re.split(r"[,\s]+", selector.strip()):
        if not tok: continue
        m = re.fullmatch(r"(\d{3})-(\d{3})", tok)
        if m:
            a, b = int(m.group(1)), int(m.group(2))
            if a <= b: keep.update(range(a, b + 1))
            else:      keep.update(range(b, a + 1))
            continue
        m2 = re.fullmatch(r"(\d{3})", tok)
        if m2:
            keep.add(int(m2.group(1)))
    keep = {x for x in keep if 1 <= x <= 500}
    return keep or None

def extract_code_from_company_dirname(name: str):
    m = re.match(r"^\s*(\d{3})\b", name or "")
    return int(m.group(1)) if m else None

# ---------- Build manifest across companies for later analysis ----------
companies_root = Path(COMPANIES_ROOT)
allowed_codes = parse_folder_select(FOLDER_SELECT)

company_dirs = [d for d in companies_root.iterdir() if d.is_dir()]
print(f"Company directories found under COMPANIES_ROOT: {len(company_dirs)}")

frames = []
manifests_found = 0
checked = 0

for cdir in sorted(company_dirs):
    checked += 1
    code = extract_code_from_company_dirname(cdir.name)
    if code is None:
        continue
    if allowed_codes is not None and code not in allowed_codes:
        continue

    stage_dir = None
    mpath = None
    for candidate in ("esg_stage", "esg_stage1"):
        cand_dir = cdir / candidate
        cand_manifest = cand_dir / "manifest.parquet"
        if cand_manifest.exists():
            stage_dir = cand_dir
            mpath = cand_manifest
            break
    if stage_dir is None:
        continue

    try:
        df = pd.read_parquet(mpath)
    except Exception as e:
        print(f"[WARN] Could not read manifest for {cdir.name}: {e}")
        continue

    df = df[df["status"].isin(["ok_native","ok_ocr","ok_hybrid","ok"])].copy()
    if df.empty:
        continue

    df["company_code"] = code
    df["__base_dir__"] = str(stage_dir.resolve())  # points to company/esg_stage* (for _resolve)
    df["__company_dir__"] = str(cdir.resolve())    # points to company/ (for output dir esg_stage2)
    frames.append(df)
    manifests_found += 1

print(f"Companies scanned (after name check & FOLDER_SELECT): {checked}")
print(f"Manifests found: {manifests_found}")

if not frames:
    raise FileNotFoundError(
        "No manifest.parquet found under selected companies. "
        "Checked both 'esg_stage' and 'esg_stage1'. "
        "Verify COMPANIES_ROOT path and folder names like '001. Walmart'."
    )

man = pd.concat(frames, ignore_index=True)

if YEAR_LIST:   man = man[man["doc_year"].isin(YEAR_LIST)]
if REPORT_TYPE: man = man[man["report_type"].isin(REPORT_TYPE)]
man = man.reset_index(drop=True)

print(f"Companies selected: {sorted(set(man['company_code']))}")
print(f"After selection, docs to process: {len(man)}")

# ---------- Output path: save JSON in company/esg_stage2/ ----------
def out_path(row):
    company_dir = Path(row.get("__company_dir__", ""))        # .../FORTUNE 500/001. Walmart
    out_dir = company_dir / "esg_stage2"                      # sibling of esg_stage / esg_stage1
    did = (
        str(row["sha256_16"])
        if pd.notna(row.get("sha256_16"))
        else Path(str(row.get("text_path") or row.get("pdf_path") or row.get("filename") or "doc")).stem
    )
    out_dir.mkdir(parents=True, exist_ok=True)
    return out_dir / f"esg_stage2_{did}.json"

# ---------- SIC cache: classify once per company, reuse ----------
SIC_CACHE = {}  # { company_code: (div_code, div_name) }

def get_company_sic(row):
    code = int(row.get("company_code")) if pd.notna(row.get("company_code")) else None
    if code in SIC_CACHE:
        return SIC_CACHE[code]
    # pick this row as the sample doc for the company
    sents = read_sentences(row.to_dict())
    if not sents:
        # fallback: assume Services if empty
        SIC_CACHE[code] = ("I", "Services")
        return SIC_CACHE[code]
    div_code, div_name, _ = classify_division(sents)
    SIC_CACHE[code] = (div_code, div_name)
    return SIC_CACHE[code]

# ---------- Main loop ----------
proc=skip=0
for _,row in man.iterrows():
    op = out_path(row)
    if ONLY_MISSING and op.exists():
        skip += 1
        continue

    # company-level SIC (computed once per company)
    pre_sic = get_company_sic(row)  # tuple (div_code, div_name)

    sents = read_sentences(row.to_dict())
    res   = analyze_doc(sents, precomputed_div=pre_sic)
    res["meta"] = {
        "filename": row.get("filename"),
        "doc_year": int(row.get("doc_year")) if pd.notna(row.get("doc_year")) else None,
        "report_type": row.get("report_type"),
        "company_code": int(row.get("company_code")) if pd.notna(row.get("company_code")) else None,
        "company_dir": row.get("__company_dir__", ""),
        "stage1_dir": row.get("__base_dir__", "")
    }
    with open(op,"w",encoding="utf-8") as f: json.dump(res,f,ensure_ascii=False,indent=2)
    proc+=1
    print(f"Processed {proc}: wrote {op}")

print(f"Done. Wrote {proc}; skipped {skip}. Results saved in each company/esg_stage2/")


Mounted at /content/drive
Company directories found under COMPANIES_ROOT: 501
Companies scanned (after name check & FOLDER_SELECT): 501
Manifests found: 5
Companies selected: [6, 7, 8, 9, 10]
After selection, docs to process: 28
Processed 1: wrote /content/drive/MyDrive/Australia Award Scholarship/USYD/Dissertation/DATA/FORTUNE 500/006. ExxonMobil/esg_stage2/esg_stage2_f125df312bf1444f.json
Processed 2: wrote /content/drive/MyDrive/Australia Award Scholarship/USYD/Dissertation/DATA/FORTUNE 500/006. ExxonMobil/esg_stage2/esg_stage2_8910827c98147d81.json
Processed 3: wrote /content/drive/MyDrive/Australia Award Scholarship/USYD/Dissertation/DATA/FORTUNE 500/006. ExxonMobil/esg_stage2/esg_stage2_93b3ce40633fe859.json
Processed 4: wrote /content/drive/MyDrive/Australia Award Scholarship/USYD/Dissertation/DATA/FORTUNE 500/006. ExxonMobil/esg_stage2/esg_stage2_faafd0ad9425bfb4.json
Processed 5: wrote /content/drive/MyDrive/Australia Award Scholarship/USYD/Dissertation/DATA/FORTUNE 500/006. E